In [115]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import json
import fileinput
from tqdm import tqdm
import string
import re
import json
from os import walk
import os
import ast
import re

data_dir = "/Users/jaeyoungkim/Desktop/usc/DSCI - 558/project/sub/structured_data/"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [116]:
tot_game_df = pd.DataFrame()

In [117]:
f = []
for (dirpath, dirnames, filenames) in walk(data_dir):
    f.extend(filenames)

In [118]:
for file in f:
    if file[0] == "s":
        df = pd.read_csv(data_dir + file)
        tot_game_df = tot_game_df.append(df)

In [119]:
tot_game_df      

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,LBA,SJH,SJD,SJA,BSH,BSD,BSA,SBH,SBD,SBA
0,E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,11/08/2018,Huddersfield,Chelsea,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,11/08/2018,Newcastle,Tottenham,1,2,A,1,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2010-05-09,Everton,Portsmouth,1,0,H,0,0,D,...,10.00,1.29,5.50,11.00,1.30,5.00,10.00,1.29,5.00,8.50
376,E0,2010-05-09,Hull,Liverpool,0,0,D,0,0,D,...,1.57,6.50,3.75,1.57,6.00,3.75,1.57,6.00,3.75,1.50
377,E0,2010-05-09,Man United,Stoke,4,0,H,2,0,H,...,19.00,1.11,9.00,21.00,1.11,7.50,23.00,1.11,7.50,15.00
378,E0,2010-05-09,West Ham,Man City,1,1,D,1,1,D,...,1.80,3.75,3.60,1.91,4.20,3.60,1.80,4.00,3.40,1.80


In [120]:
wiki_team = pd.DataFrame()
with fileinput.input(files="wikipedia_crawler/team_info_wikipedia.jl") as file:
    for line in file:
        conv = json.loads(line)
        wiki_team = wiki_team.append(conv, ignore_index=True)
wiki_team.head()

,wiki_team_url,name,Fullname,Nickname(s),Founded,Ground,Capacity,Owner,Chairman,Manager,...,HeadCoach,CEO,Owner(s),Joint-Chairman,Co-chairmen,Owners,Stadium,Anear-fullcapacity,Location,Opened
0,https://en.wikipedia.org/wiki/Wigan_Athletic,Wigan Athletic,[WiganAthleticFootballClub],"[TheLatics, TheTics]","[1932, ;89 yearsago, (, 1932, )]",[DWStadium],25138,[Phoenix2021Limited],TalalAlHammad,[LeamRichardson],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.wikipedia.org/wiki/Tottenham_Hotspur,Tottenham Hotspur,[TottenhamHotspurFootballClub],[TheLilywhites],"[5 September1882, ;139yearsago, (, 1882-09-05...",[TottenhamHotspurStadium],62850,"[ENICInternationalLtd., (85.55%)]",DanielLevy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Birmingham_City,Birmingham City F.C.,[BirminghamCityFootballClub],[Blues],"[1875, ;146 yearsago, (, 1875, ), as, SmallHe...",[StAndrew's],29409,"[BirminghamSportsHoldings75.00%, OrientalRainb...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Wolverhampton_Wa...,Wolverhampton Wanderers,[WolverhamptonWanderersFootballClub],[WolvesTheWanderers],"[1877, ;144 yearsago, (, 1877, ), as, St.Luke...",[MolineuxStadium],32050,[FosunInternational],JeffShi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Sunderland_AFC,Sunderland A.F.C.,[SunderlandAssociationFootballClub],[TheBlackCats],"[1879, ;142 yearsago, (, 1879, ), (as, Sunder...",[StadiumofLight],49000,"[KyrilLouis-Dreyfus, (majority)]",KyrilLouis-Dreyfus,NaN,...,[LeeJohnson],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
world_team = pd.DataFrame()
with fileinput.input(files="worldfootball_crawler/team_info_worldfootball.jl") as file:
    for line in file:
        conv = json.loads(line)
        world_team = world_team.append(conv, ignore_index=True)
world_team.head()

,name,world_team_url,Country,Founded,stadium,Homepage
0,Liverpool FC,https://www.worldfootball.net/teams/liverpool-fc/,[England],[15/03/1892],[Anfield],[www.liverpoolfc.com/]
1,Tottenham Hotspur,https://www.worldfootball.net/teams/tottenham-...,[England],[05/09/1882],[TottenhamHotspurStadium],[www.spurs.co.uk]
2,Sunderland AFC,https://www.worldfootball.net/teams/sunderland...,[England],[01/01/1879],[StadiumofLight],[www.safc.com]
3,Manchester United,https://www.worldfootball.net/teams/manchester...,[England],[01/03/1878],[OldTrafford],[www.manutd.com/]
4,Wigan Athletic,https://www.worldfootball.net/teams/wigan-athl...,[England],[01/01/1932],[DWStadium],[https//www.wiganathletic.com/]


In [122]:
who_team = pd.DataFrame()
with fileinput.input(files="whoscored_crawler/team_whoscored.jl") as file:
    for line in file:
        conv = json.loads(line)
        who_team = who_team.append(conv, ignore_index=True)
who_team.head()

,season,name,goals,team_url,shots_per_game,yellow,red,possessions_per,pass_per,Aerialswon,Rating
0,2009/2010,Chelsea,103,https://www.whoscored.com/Teams/15/Show/Englan...,21.9,54,5,56.9,82.5,11.6,7.30
1,2009/2010,Tottenham,67,https://www.whoscored.com/Teams/30/Show/Englan...,17.9,58,1,52.2,76.1,18.2,7.19
2,2009/2010,Manchester United,86,https://www.whoscored.com/Teams/32/Show/Englan...,18.3,49,4,55.6,79.9,13.2,7.19
3,2009/2010,Arsenal,83,https://www.whoscored.com/Teams/13/Show/Englan...,17.4,56,1,57.0,80.8,14.4,7.17
4,2009/2010,Aston Villa,52,https://www.whoscored.com/Teams/24/Show/Englan...,13.1,59,2,48.5,71.8,18.8,7.06


In [123]:
wiki_team["Fullname"] = wiki_team["Fullname"].apply(lambda  x : x[0])

In [124]:
def f(x):
    for idx1, item in enumerate(x):
        for idx, month in enumerate(["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]):
            x[idx1] = re.sub(month, "-" + str(idx+1) + "-", x[idx1]).split(";")[0]
    r = re.compile("\d{1,4}-\d{1,2}-\d{1,4}|\d{4}$|\d{2}-\d{2}-\d{4}|\d{2}-\d{1}-\d{4}|\d{2}-\d{4}")
    filtered_list = list(filter(r.match, x))
    return filtered_list

In [125]:
wiki_team['Founded'] = wiki_team["Founded"].apply(lambda x : set(f([i.lower() for i in x])))

In [126]:
Lev = NormalizedLevenshtein().similarity

In [127]:
wiki_name2idx = dict()
wiki_idx2name = dict()
for idx, name in enumerate(wiki_team["name"]):
    wiki_name2idx[name] = idx
    wiki_idx2name[idx] = name

In [128]:
def f(x):
    tmp = [0,0]
    for name in wiki_team["name"]:
        if Lev(x, name) > 0.3 and Lev(x, name) > tmp[1]:
            tmp[0] = wiki_name2idx[name]
            tmp[1] = Lev(x,name)
    if tmp != [0,0]:
        return tmp[0]

who_team["id"] = who_team["name"].apply(f)
world_team["id"] = world_team["name"].apply(f)


In [131]:
wiki_team["id"] = wiki_team["name"].apply(lambda x : wiki_name2idx[x])

In [132]:
who_team.loc[who_team.name == "Bolton", "id"] = 18

who_team.loc[who_team.name == "Brighton", "id"] = 35

In [133]:
team_merged = wiki_team.merge(world_team, left_on = "id", right_on = 'id', how = "outer")
team_merged = team_merged.merge(who_team[["name", "team_url", "id"]].drop_duplicates().rename({"team_url" : "who_team_url"}, axis = 1), left_on = "id", right_on = 'id', how = "outer")
team_merged.to_csv("team_join.csv")

In [134]:
team_merged

,wiki_team_url,name_x,Fullname,Nickname(s),Founded_x,Ground,Capacity,Owner,Chairman,Manager,...,Opened,id,name_y,world_team_url,Country,Founded_y,stadium,Homepage,name,who_team_url
0,https://en.wikipedia.org/wiki/Wigan_Athletic,Wigan Athletic,WiganAthleticFootballClub,"[TheLatics, TheTics]",{1932},[DWStadium],25138,[Phoenix2021Limited],TalalAlHammad,[LeamRichardson],...,NaN,0,Wigan Athletic,https://www.worldfootball.net/teams/wigan-athl...,[England],[01/01/1932],[DWStadium],[https//www.wiganathletic.com/],Wigan,https://www.whoscored.com/Teams/194/Show/Engla...
1,https://en.wikipedia.org/wiki/Tottenham_Hotspur,Tottenham Hotspur,TottenhamHotspurFootballClub,[TheLilywhites],{1882-09-05},[TottenhamHotspurStadium],62850,"[ENICInternationalLtd., (85.55%)]",DanielLevy,NaN,...,NaN,1,Tottenham Hotspur,https://www.worldfootball.net/teams/tottenham-...,[England],[05/09/1882],[TottenhamHotspurStadium],[www.spurs.co.uk],Tottenham,https://www.whoscored.com/Teams/30/Show/Englan...
2,https://en.wikipedia.org/wiki/Birmingham_City,Birmingham City F.C.,BirminghamCityFootballClub,[Blues],{1875},[StAndrew's],29409,"[BirminghamSportsHoldings75.00%, OrientalRainb...",NaN,NaN,...,NaN,2,Birmingham City,https://www.worldfootball.net/teams/birmingham...,[England],[01/01/1875],[StAndrew'sGround],[www.bcfc.com],Birmingham,https://www.whoscored.com/Teams/157/Show/Engla...
3,https://en.wikipedia.org/wiki/Wolverhampton_Wa...,Wolverhampton Wanderers,WolverhamptonWanderersFootballClub,[WolvesTheWanderers],{1877},[MolineuxStadium],32050,[FosunInternational],JeffShi,NaN,...,NaN,3,Wolverhampton Wanderers,https://www.worldfootball.net/teams/wolverhamp...,[England],[01/01/1877],[MolineuxStadium],[https//www.wolves.co.uk/],Wolverhampton Wanderers,https://www.whoscored.com/Teams/161/Show/Engla...
4,https://en.wikipedia.org/wiki/Sunderland_AFC,Sunderland A.F.C.,SunderlandAssociationFootballClub,[TheBlackCats],{1879},[StadiumofLight],49000,"[KyrilLouis-Dreyfus, (majority)]",KyrilLouis-Dreyfus,NaN,...,NaN,4,Sunderland AFC,https://www.worldfootball.net/teams/sunderland...,[England],[01/01/1879],[StadiumofLight],[www.safc.com],Sunderland,https://www.whoscored.com/Teams/16/Show/Englan...
5,https://en.wikipedia.org/wiki/Liverpool_FC,Liverpool,LiverpoolFootballClub,[TheReds],{1892-06-03},[Anfield],53394,[FenwaySportsGroup],TomWerner,[JürgenKlopp],...,NaN,5,Liverpool FC,https://www.worldfootball.net/teams/liverpool-fc/,[England],[15/03/1892],[Anfield],[www.liverpoolfc.com/],Liverpool,https://www.whoscored.com/Teams/26/Show/Englan...
6,https://en.wikipedia.org/wiki/Fulham_FC,Fulham,FulhamFootballClub,[TheCottagersTheWhitesTheLilyWhites],{1879},[CravenCottage],19359,[ShahidKhan],ShahidKhan,NaN,...,NaN,6,Fulham FC,https://www.worldfootball.net/teams/fulham-fc/,[England],[01/01/1879],[CravenCottage],[www.fulhamfc.com],Fulham,https://www.whoscored.com/Teams/170/Show/Engla...
7,https://en.wikipedia.org/wiki/Manchester_City,Manchester City,ManchesterCityFootballClub,"[CityCityzens, TheCitizensTheSkyBlues]","{1880, 1894-04-16, 1887}",[CityofManchesterStadium],53400,[CityFootballGroup],KhaldoonAlMubarak,[PepGuardiola],...,NaN,7,Manchester City,https://www.worldfootball.net/teams/manchester...,[England],[01/01/1880],[EtihadStadium],[https//www.mancity.com/],Manchester City,https://www.whoscored.com/Teams/167/Show/Engla...
8,https://en.wikipedia.org/wiki/Blackburn_Rovers,Blackburn Rovers,BlackburnRoversFootballClub,"[Rovers, TheBlueandWhites, TheRiversiders]",{1875},[EwoodPark],31367,"[Venky'sLtd., (99.9%)]",NaN,[TonyMowbray],...,NaN,8,Blackburn Rovers,https://www.worldfootball.net/teams/blackburn-...,[England],[05/11/1875],[EwoodPark],[https//www.rovers.co.uk/],Blackburn,https://www.whoscored.com/Teams/158/Show/Engla...
9,https://en.wikipedia.org/wiki/Aston_Villa,Aston Villa,AstonVillaFootballClub,"[TheVilla, TheLions, TheClaret&BlueArmy]",{1874-11-21},[VillaPark],42749,NaN,NassefSawiris,NaN,...,NaN,9,Aston Villa,https://www.worldfootball.net/teams/aston-villa/,[England],[03/1874],[Vil

In [135]:
match = team_merged[["wiki_team_url", "world_team_url", "who_team_url"]]
match = match.append({"wiki_team_url" : 'https://en.wikipedia.org/wiki/Brentford_F.C.', "world_team_url" : 'https://www.worldfootball.net/teams/brentford-fc/', "who_team_url" : 'https://www.whoscored.com/Teams/189/Show/England-Brentford'}, ignore_index = True)
match["id"] = [i for i in range(len(match))]

In [136]:
match.loc[38,"who_team_url"] = 'https://www.whoscored.com/Teams/211/Show/England-Brighton'
match = match.drop(35, axis = 0)

In [ ]:
match.to_csv("team_match.csv", index = None)